# Modes of a Vibrating Building

In this notebook we will find the vibrational modes of a simple model of a building. We will assume that the mass of the floors are much more than the mass of the walls and that the lateral stiffness of the walls can be modeled by a simple linear spring. We will investigate how the building may vibrate under initial conditions that could be caused by a gust of wind and during ground vibration.

In [ ]:
from IPython.display import YouTubeVideo

In [ ]:
YouTubeVideo('pMr1MzSv044', width=600)

In [ ]:
YouTubeVideo('hSwjkG3nv1c', width=600)

In [ ]:
YouTubeVideo('kzVvd4Dk6sw', width=600)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from resonance.linear_systems import FourStoryBuildingSystem

This gives a bit nicer printing of large NumPy arrays.

In [ ]:
np.set_printoptions(precision=5, linewidth=100, suppress=True)

In [ ]:
%matplotlib widget

# Simulate the free response of the four story building

In [ ]:
sys = FourStoryBuildingSystem()

In [ ]:
sys.constants

In [ ]:
sys.coordinates

In [ ]:
sys.plot_configuration();

In [ ]:
traj = sys.free_response(30, sample_rate=10)

In [ ]:
traj[sys.coordinates.keys()].plot(subplots=True, sharey=True);

In [ ]:
sys.animate_configuration(fps=10, repeat=False)

# Extract the mass, damping, and stiffness matrices

In [ ]:
M, C, K = sys.canonical_coefficients()

In [ ]:
M

In [ ]:
C

In [ ]:
K

# Exercise

$$
\mathbf{M}\dot{\bar{s}} + \mathbf{K}\bar{c} = \bar{0}
$$

Since the mass and stiffness matrices are symmetric and there is no damping, the system can be normalized by the mass matrix and transformed into a symmetric eigenvalue problem by introducing the new coordinate vector:

$$\bar{q}=\mathbf{L}^T\bar{c}$$

$\mathbf{L}$ is the Cholesky decomposition of the symmetric mass matrix, i.e. $\mathbf{M}=\mathbf{L}\mathbf{L}^T$.

The equation of motion becomes:

$$\ddot{\bar{q}} + \tilde{\mathbf{K}} \bar{q} = 0$$

Compute $\tilde{\mathbf{K}}$.

In [ ]:
import numpy.linalg as la

In [ ]:
L = np.linalg.cholesky(M)
K_tilde = la.inv(L) @ K @ la.inv(L.T)
K_tilde

In [ ]:
# write your answer here

Notice that $\tilde{\mathbf{K}}$ is symmetric, so we are guaranteed to get real eigenvalues and orthogonal eigenvectors when solving this system.

# Exercise

Find the eigenvalues and eigenvectors. Create the spectral matrix $\mathbf{\Lambda}$ and the matrix $\mathbf{P}$ which contains the orthonormal eigenvectors of $\tilde{\mathbf{K}}$. Store in variables `Lambda` and `P` respectively.

$$
\mathbf{P} = \left[ \hat{q}_{01}, \ldots, \hat{q}_{04} \right]
$$

In [ ]:
evals, evecs = np.linalg.eig(K_tilde)
Lambda = np.diag(evals)
print(Lambda)
P = evecs
print(P)

In [ ]:
# write your answer here

# Exercise

Prove that the eigenvectors in $\mathbf{P}$ are orthonormal.

In [ ]:
np.dot(P[:, 0], P[:, 1])

np.linalg.norm(P[:, 0])

P[:, 0].T @ P[:, 1]

P[:, 0].T @ P[:, 0]

In [ ]:
# write your answer here

An orthonormal matrix has the property that its transpose multiplied by itself is the identity matrix.

$$
\mathbf{P}^T\mathbf{P} = \mathbf{I}
$$

In [ ]:
P.T @ P

# Exercise

Find the natural freqencies of the system in both radians per second and Hertz, store them in an array in the order of the eigenvalues with names `ws` and `fs`.

In [ ]:
ws = np.sqrt(evals)
ws

In [ ]:
fs = ws / 2 / np.pi
fs

In [ ]:
# write your answer here

# Exercise

Transform the eigenvectors back into the coordinate system associated with the coordinates $\bar{c}$. Store the results in variable `S` representing the matrix below.

$$
\mathbf{S} = \left[ \bar{u}_1, \ldots, \bar{u}_4 \right]
$$

In [ ]:
S = np.linalg.inv(L.T) @ P
S

In [ ]:
# write your answer here

# Visualize the mode shapes

In [ ]:
YouTubeVideo('diHuLKYfTXM', width=600)

The eigenmodes (mode shapes) are contained in each column of $\mathbf{S}$. The plot will have these specifications:

- The title of each plot should be the frequency of the corresponding modeshape in Hz.
- The y axis should be made up of the values [0, 3, 6, 9, 12] meters.
- The x axis should plot the five values. The first should be zero and the remaining values should be the components of the mode shape in order of the component associated with the lowest floor to the highest.
- Plot lines with small circles at each data point.

In [ ]:
fig, axes = plt.subplots(1, 4, sharey=True)

for i in range(4):
    axes[i].grid()
    axes[i].plot(np.hstack((0, S[:, i])),  # add a zero so the line connects to the ground
                 [0, 3, 6, 9, 12],  # arbitrary floor heights
                 marker='o')
    axes[i].set_title('{:1.2f} Hz'.format(fs[i]))
    axes[i].set_xlim((-0.015, 0.015))
    
plt.tight_layout()

The modes can be simulated by setting the initial coordinates equal to the corresponding vector $\bar{u}_i$. For example, here is 3rd mode shape:

In [ ]:
sys.coordinates['x1'] = S[0, 2]
sys.coordinates['x2'] = S[1, 2]
sys.coordinates['x3'] = S[2, 2]
sys.coordinates['x4'] = S[3, 2]

In [ ]:
traj = sys.free_response(10, sample_rate=10)

In [ ]:
traj[sys.coordinates.keys()].plot(subplots=True, sharey=True)  # note the sharey

In [ ]:
sys.animate_configuration(fps=10, repeat=False)

Explore the different mode shape animations with the widget:

In [ ]:
def animate_building(mode_num=0):
    sys.coordinates['x1'] = S[0, mode_num]
    sys.coordinates['x2'] = S[1, mode_num]
    sys.coordinates['x3'] = S[2, mode_num]
    sys.coordinates['x4'] = S[3, mode_num]
    traj = sys.free_response(10, sample_rate=30)
    ani = sys.animate_configuration(fps=10, repeat=False)
    return ani

In [ ]:
from ipywidgets import interact

interact(animate_building, mode_num=[0, 1, 2, 3])

# Simulating the trajectory via the ODE solution

The trajectory of building's coordinates can be found with the general solution to the ODEs:

$$
\bar{c}(t) = \sum_{i=1}^n d_i \sin(\omega_i t + \phi_i) \bar{u}_i
$$

where

$$
\phi_i = \arctan \frac{\omega_i \hat{q}_{0i}^T \bar{q}(0)}{\hat{q}_{0i}^T \dot{\bar{q}}(0)}
$$

and

$$
d_i = \frac{\hat{q}_{0i}^T \bar{q}(0)}{\sin\phi_i}
$$

$d_i$ are the modal participation factors and reflect what proportion of each mode is excited given specific initial conditions. If the initial conditions are the eigenmode, $\bar{u}_i$, the all but the $i$th $d_i$ will be zero.

# Exercise

Show that if $\bar{q}(0) = \bar{u}_i$ then $d_i = 1$ all other modal participation factors are 0. Also, report all of the phase angles, $\phi_i$, in degrees. The following function should calcualtes all the $\phi$'s and $d$'s given arbitrary initial conditions.

```python
def calc_phis_ds(c0, s0, ws, L, P):
    """
    Parameters
    ==========
    c0 : ndarray, shape(n,)
        Initial coordinates: x1, x2, x3, x4.
    s0 : ndarray, shape(n,)
        Initial speeds: v1, v2, v3, v4.
    ws : ndarray, shape(n,)
        The eigenfrequencies in radians per second.
    P : ndarray, shape(n, n)
        The matrix of orthnormal eigenvectors. Columns correspond to the order of ``ws`.
    L : ndarray, shape(n, n)
        The Cholesky decomposition matrix of the mass matrix M.
    
    Returns
    =======
    phis : ndarray, shape(n,)
        The phase shift angles in radians corresponding to each mode.
    ds : ndarray, shape(n,)
        The participation factors corresponding to each mode.
    
    """
    
    q0 = L.T @ c0
    qd0 = L.T @ s0

    phis = # fill out this line

    ds = # fill out this line

    return phis, ds
```

In [ ]:
def calc_phis_ds(c0, s0, ws, L, P):
    """
    Parameters
    ==========
    c0 : ndarray, shape(n,)
        Initial coordinates: x1, x2, x3, x4.
    s0 : ndarray, shape(n,)
        Initial speeds: v1, v2, v3, v4.
    ws : ndarray, shape(n,)
        The eigenfrequencies in radians per second.
    P : ndarray, shape(n, n)
        The matrix of orthnormal eigenvectors. Columns correspond to the order of ``ws`.
    L : ndarray, shape(n, n)
        The Cholesky decomposition matrix of the mass matrix M.
    
    Returns
    =======
    phis : ndarray, shape(n,)
        The phase shift angles in radians corresponding to each mode.
    ds : ndarray, shape(n,)
        The participation factors corresponding to each mode.
    
    """
    
    q0 = L.T @ c0
    qd0 = L.T @ s0

    phis = np.arctan2(ws * P.T @ q0, P.T @ qd0)

    ds = P.T @ q0 / np.sin(phis)

    return phis, ds

In [ ]:
print(calc_phis_ds(np.random.random(4), np.zeros(4), ws, L, P))

print(calc_phis_ds(S[:, 0], np.zeros(4), ws, L, P))

print(calc_phis_ds(S[:, 1], np.zeros(4), ws, L, P))

print(calc_phis_ds(S[:, 2], np.zeros(4), ws, L, P))

print(calc_phis_ds(S[:, 3], np.zeros(4), ws, L, P))

In [ ]:
# write answer here

# Exercise

How do the $\phi$'s and $d$'s change if the initial coordinates are set to one of the $\bar{u}$'s but the initial speeds are set to arbitrary values?

In [ ]:
calc_phis_ds(S[:, 3], np.random.random(4), ws, L, P)

In [ ]:
# write you answer here

# Simulation

The following function simulates the system given a time array and initial conditions.

In [ ]:
def simulate(t, c0, s0, ws, L, P, S):
    """
    Parameters
    ==========
    t : ndarray, shape(m,)
        Monotonically increasing values of time.
    c0 : ndarray, shape(n,)
        Initial coordinates: x1, x2, x3, x4.
    s0 : ndarray, shape(n,)
        Initial speeds: v1, v2, v3, v4.
    ws : ndarray, shape(n,)
        The eigenfrequencies in radians per second.
    P : ndarray, shape(n, n)
        The matrix of orthnormal eigenvectors. Columns correspond to the order of ``ws`.
    L : ndarray, shape(n, n)
        The Cholesky decomposition matrix of the mass matrix M.
    S : ndarray, shape(n, n)
        The matrix of eigenvectors projected back into the coordinate space, c.
    
    Returns
    =======
    c : ndarray, shape(n, m)
        The coordinates as a function of time.
    
    """
    
    phis, ds = calc_phis_ds(c0, s0, ws, L, P)
    
    c = np.zeros((len(c0), len(t)))
    for di, wi, phii, ui in zip(ds, ws, phis, S.T):
        c += di * np.sin(wi * t + phii) * np.tile(ui, (len(t), 1)).T
    
    return c

Use the `free_response()` function to obtain trajectories with the initial conditions set to an arbitrary value.

In [ ]:
sys.coordinates['x1'] = 0.001
sys.coordinates['x2'] = 0.010
sys.coordinates['x3'] = 0.020
sys.coordinates['x4'] = 0.025

traj = sys.free_response(30, sample_rate=30)

axes = traj[['x1', 'x2', 'x3', 'x4']].plot(subplots=True, linewidth=4.0)

# Exercise

Plot the trajectories of the coordinates from the `simulate()` function on top of the lines produced from `free_response()`. Make them black with a linestyle of `--`. You can use `axes[0].plot(...)` to plot on top of the existing lines.

In [ ]:
t = np.linspace(0, 30, num=30 * 30)
c0 = np.array([0.001, 0.010, 0.020, 0.025])
s0 = np.zeros(4)
c = simulate(t, c0, s0, ws, L, P, S)

axes[0].plot(t, c[0, :], color='black', linestyle='--')
axes[1].plot(t, c[1, :], color='black', linestyle='--')
axes[2].plot(t, c[2, :], color='black', linestyle='--')
axes[3].plot(t, c[3, :], color='black', linestyle='--')

In [ ]:
# write your answer here